In [4]:
import boto3
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import tifffile
import math
import consts
from shapely.geometry import shape

matplotlib.use('TkAgg')
s3 = boto3.resource('s3')
os.makedirs(consts.PATH, exist_ok=True)

def download_file(s3_bucket, s3_key, local_path):
	try:
		s3_bucket.download_file(s3_key, local_path)
		return local_path
	except Exception as e:
		print(e)
			
def overlay_cloud_mask(image, cloud_mask=None, shadow_mask=None, figsize=(10, 10), fig=None):
	"""
	Utility function for plotting RGB images with binary mask overlayed.
	"""
	if fig is None:
		plt.figure(figsize=figsize)
	plt.imshow(image)
	if cloud_mask is not None:
		cloud_image = np.zeros((cloud_mask.shape[0], cloud_mask.shape[1], 4), dtype=np.uint8)
		cloud_image[cloud_mask == True] = np.asarray([255, 255, 0, 100], dtype=np.uint8)
		plt.imshow(cloud_image)
	if shadow_mask is not None:
		cloud_image = np.zeros((shadow_mask.shape[0], shadow_mask.shape[1], 4), dtype=np.uint8)
		cloud_image[shadow_mask == True] = np.asarray([255, 204, 229, 100], dtype=np.uint8)
		plt.imshow(cloud_image)
	plt.show(block=True)
		

In [68]:
bucket = s3.Bucket(consts.BUCKET_NAME)
items = []
for key in bucket.objects.all():
	if len(items) >= 50:
		break
	if consts.ATMOSPHERIC_CORRECTION in key.key and key.key.endswith('.tif'):
		items.append(key.key[:key.key.rfind('/') + 1])

direc = []
for item in items:
	temp = []
	for key in bucket.objects.filter(Prefix=item):
		if key.key.endswith('.tif') or key.key.endswith('.json'):
			temp.append(key.key)
	if len(temp) == 2:
		direc.append(temp)

file_paths = []
for folder in direc:
	last = folder[0].rfind('/')
	folder_path = os.path.join(consts.PATH, folder[0][folder[0].rfind('/', 0, last) + 1:last])
	os.makedirs(folder_path, exist_ok=True)
	for file in folder:
		file_path = os.path.join(folder_path, file[file.rfind('/') + 1:])
		download_file(bucket, file, file_path)
		file_paths.append(file_path)

In [5]:
file_path = consts.TIF_FILE_PATH
metafile = open(os.path.splitext(file_path)[0] + '_metadata.json')
metadata = json.load(metafile)
sky_meta = metadata['Metadata']['SkyWatch_Metadata']
sun_elev, sun_azim = sky_meta['sun_elevation'], sky_meta['sun_azimuth']

img_array = tifffile.imread(file_path)
img_trans = np.transpose(img_array[0:3], (1, 2, 0))

red, green, blue, nir, flags = img_array[0:5]
rxgxb = np.multiply(np.multiply(red, green), blue)
rdivgdivb = np.divide(np.divide(red, green, out=np.zeros_like(red), where=green!=0), blue, out=np.zeros_like(red), where=blue!=0)
ndvi = np.divide(np.subtract(nir, red), np.add(nir, red))

rxgxb = rxgxb.ravel()
rdivgdivb = rdivgdivb.ravel()
ndvi = ndvi.ravel()
flags = flags.ravel()
flags = [f'{int(flag):013b}'[-5] == '1' for flag in flags]

cloud_mask = np.ma.masked_where(flags | ((rxgxb > 0.002) & (rdivgdivb < 5.9) & ((ndvi < 0.5) & (ndvi > -0.04))), rxgxb).mask
cloud_mask = cloud_mask.reshape(red.shape)

# CLOUD SHADOW MASKING
red = red.ravel()
green = green.ravel()
blue = blue.ravel()
filtered_mask = np.ma.masked_where((red < 0.1) & (green < 0.1) & (blue < 0.1) & (ndvi < 0.5), rxgxb).mask
filtered_mask = filtered_mask.reshape(cloud_mask.shape)

def update_shadow_mask(new_i, new_j):
	shadow_mask[new_i, new_j] = True
		
def valid_pixel(new_i, new_j):
	if not(0 <= new_i < shadow_mask.shape[0] and 0 <= new_j < shadow_mask.shape[1]):
		return False
	if cloud_mask[new_i, new_j] or shadow_mask[new_i, new_j]:
		return False
	return True
		
def handle_bot_left_check(new_i, new_j):
	new_j -= 1
	if not valid_pixel(new_i, new_j):
		return
	point = shape({
		"type": "Point",
		"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
		update_shadow_mask(new_i, new_j)
		handle_bot_left_check(new_i, new_j)
	else:
		return
		
	new_i += 1
	if not valid_pixel(new_i, new_j):
		return
	point = shape({
		"type": "Point",
		"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
		update_shadow_mask(new_i, new_j)
		handle_bot_left_check(new_i, new_j)
	else:
		return
	
	new_j += 1
	if not valid_pixel(new_i, new_j):
		return
	point = shape({
		"type": "Point",
		"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
		update_shadow_mask(new_i, new_j)
		handle_bot_left_check(new_i, new_j)
	else:
		return
		
		
def handle_top_left_check(new_i, new_j):
	new_j -= 1
	point = shape({
		"type": "Point",
    	"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
		
	new_i -= 1
	point = shape({
    	"type": "Point",
    	"coordinates": [new_i, new_j]
    })
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
	
	new_j += 1
	point = shape({
		"type": "Point",
		"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
		
		
def handle_top_right_check(new_i, new_j):
	new_i -= 1
	point = shape({
    	"type": "Point",
       	"coordinates": [new_i, new_j]
    })
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
	
	new_j += 1
	point = shape({
       	"type": "Point",
       	"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
	
	new_i += 1
	point = shape({
		"type": "Point",
    	"coordinates": [new_i, new_j]
    })
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
		
		
def handle_bot_right_check(new_i, new_j):
	new_j += 1
	point = shape({
    	"type": "Point",
       	"coordinates": [new_i, new_j]
    })
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
	
	new_i += 1
	point = shape({
       	"type": "Point",
       	"coordinates": [new_i, new_j]
	})
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)
	
	new_j -= 1
	point = shape({
		"type": "Point",
    	"coordinates": [new_i, new_j]
    })
	if line.buffer(buffer).contains(point):
    		update_shadow_mask(new_i, new_j)
    
    		handle_bot_left_check(new_i, new_j)

buffer = 5
theta_prime = math.radians(sun_azim + 90.0)
r = math.hypot(cloud_mask.shape[0], cloud_mask.shape[1]) / 32.0
shadow_mask = np.full(cloud_mask.shape, False)

if 0.0 <= sun_azim <= 90.0:
	# check left, bottom-left and bottom indices
	check = handle_bot_left_check
elif 90.0 < sun_azim <= 180.0:
	# check left, top-left and top indices
	check = handle_top_left_check
elif 180.0 < sun_azim <= 270.0:
	# check top, top-right and right indices
	check = handle_top_right_check
elif 270.0 < sun_azim <= 360.0:
	# check right, bottom-right and bottom indices
	check = handle_bot_right_check
				
for i in range(len(cloud_mask)):
	for j in range(len(cloud_mask[i])):
		if cloud_mask[i,j]:
			# recurse 
			# check 3 adjacent indices unless already identified as cloud shadow, based on angle
			line = shape({
				"type": "LineString",
				"coordinates": [[i,j], [math.ceil(i + r * math.sin(theta_prime)), math.ceil(j + r * math.cos(theta_prime))]]
			})
			
			check(i, j)
	# print(i)
				
metafile.close()

final_shadow_mask = np.ma.masked_where(shadow_mask & filtered_mask, shadow_mask).mask

overlay_cloud_mask(img_trans, cloud_mask=cloud_mask, shadow_mask=final_shadow_mask)
# overlay_cloud_mask(matplotlib.colors.rgb_to_hsv(img_trans))

#TODO: TRY TILING THE IMAGE AND THEN EVALUATING EACH TILE, COULD BE USEFUL FOR HISTOGRAMS AS WELL

"""
"""

'\n'